# Import necessary libraries

In [ ]:
from pathlib import Path
import os
import pandas as pd
from collections import Counter, deque
import time
import json
import zipfile
from collections import deque, defaultdict


# Read the data

In [ ]:
# Defining column names
col_behaviors = ['ImpressionId', 'User', 'Time', 'History', 'Impressions']
col_news = ['NewsId', 'Category', 'SubCat', 'Title', 'Abstract', 'url', 'TitleEnt', 'AbstractEnt']

# Read TSV files with Pandas
behaviors_train = pd.read_csv("data/train/behaviors.tsv", sep="\t", header=None, names=col_behaviors)
news_train = pd.read_csv("data/train/news.tsv", sep="\t", header=None, names=col_news)

behaviors_val = pd.read_csv("data/validation/behaviors.tsv", sep="\t", header=None, names=col_behaviors)
news_val = pd.read_csv("data/validation/news.tsv", sep="\t", header=None, names=col_news)

behaviors_test = pd.read_csv("data/test/behaviors.tsv", sep="\t", header=None, names=col_behaviors)
news_test = pd.read_csv("data/test/news.tsv", sep="\t", header=None, names=col_news)

# zip train and val files
behaviors_train_val = pd.concat([behaviors_train, behaviors_val])
news_train_val = pd.concat([news_train, news_val])

# Convert time column to timestamp and sort by time
behaviors_train_val['Timestamp'] = behaviors_train_val['Time'].apply(lambda x: time.mktime(time.strptime(x, "%m/%d/%Y %I:%M:%S %p")))
behaviors_train_val = behaviors_train_val.sort_values(by='Timestamp')

# Convert time column to timestamp and sort by time
behaviors_val['Timestamp'] = behaviors_val['Time'].apply(lambda x: time.mktime(time.strptime(x, "%m/%d/%Y %I:%M:%S %p")))
behaviors_val = behaviors_val.sort_values(by='Timestamp')

# Implement baseline model

In [ ]:
# Define rolling window duration: 24 hours in seconds
TIME_WINDOW = 86400

# Initialize dictionary to store per-news statistics (clicks and impressions with timestamps)
news_stats = defaultdict(lambda: {'clicks': deque(), 'impressions': deque()})

# Get timestamp of the first impression in the validation set
first_impression_time = behaviors_val.iloc[0]['Timestamp']

# Filter training/validation data: only keep interactions within the 24h window before validation starts
behaviors_train_val = behaviors_train_val[
    (behaviors_train_val['Timestamp'] >= first_impression_time - TIME_WINDOW) &
    (behaviors_train_val['Timestamp'] < first_impression_time)
]

# Populate the initial news_stats dictionary with clicks and impressions
for _, row in behaviors_train_val.iterrows():
    if row['Impressions'] != '-':
        for news in row['Impressions'].split():
            news_id, label = news.split('-')  # Separate news ID and click label
            news_stats[news_id]['impressions'].append(row['Timestamp'])  # Store impression timestamp
            if label == '1':  # If clicked, also store in click stats
                news_stats[news_id]['clicks'].append(row['Timestamp'])


def update_news_stats(current_time, past_clicked_articles, past_impressed_articles):
    """
    Maintain the rolling 24h window:
    - Remove outdated clicks/impressions
    - Add recent ones from the previous impression
    """
    for news_id in list(news_stats.keys()):
        # Remove outdated clicks
        while news_stats[news_id]['clicks'] and news_stats[news_id]['clicks'][0] < current_time - TIME_WINDOW:
            news_stats[news_id]['clicks'].popleft()
        # Remove outdated impressions
        while news_stats[news_id]['impressions'] and news_stats[news_id]['impressions'][0] < current_time - TIME_WINDOW:
            news_stats[news_id]['impressions'].popleft()
        # Remove news entry if both lists are empty
        if not news_stats[news_id]['clicks'] and not news_stats[news_id]['impressions']:
            del news_stats[news_id]

    if past_impressed_articles:
        # Unpack impression articles and timestamp
        impression_list, timestamp = past_impressed_articles

        # Add current impressions
        for news_id in impression_list:
            news_stats[news_id]['impressions'].append(timestamp)

        # Add current clicks
        for news_id in past_clicked_articles:
            news_stats[news_id]['clicks'].append(timestamp)


def rank_news(user_impressions, current_time, past_clicked_articles, past_impressed_articles):
    """
    Rank news articles by Click-Through Rate (CTR) over the past 24 hours.
    CTR = clicks / impressions
    """
    # Update rolling stats with recent data
    update_news_stats(current_time, past_clicked_articles, past_impressed_articles)

    news_rank = []
    for news_id in user_impressions:
        stats = news_stats.get(news_id, {'clicks': deque(), 'impressions': deque()})
        impressions = len(stats['impressions'])
        clicks = len(stats['clicks'])
        ctr = clicks / impressions if impressions > 0 else 0.0  # Avoid division by zero
        news_rank.append((news_id, ctr))

    # Sort news descending by CTR
    news_rank.sort(key=lambda x: x[1], reverse=True)
    return [news_id for news_id, _ in news_rank]


def rank_submission_format(user_impressions, current_time, past_clicked_articles, past_impressed_articles):
    """
    Return list of rank positions for each news article in the original impression list.
    """
    ranked_news = rank_news(user_impressions, current_time, past_clicked_articles, past_impressed_articles)
    return [ranked_news.index(news_id) + 1 for news_id in user_impressions]


def generate_prediction_file(behaviors_df, output_file="prediction.txt"):
    """
    Generate a prediction file with CTR-based news rankings for each impression event.
    """
    past_clicked_articles = []        # List of clicked articles from previous row
    past_impressed_articles = []      # Tuple: (list of all impressions, timestamp)

    with open(output_file, "w") as f:
        for _, row in behaviors_df.iterrows():
            impression_id = row['ImpressionId']
            current_time = row['Timestamp']
            impression_entries = row['Impressions'].split()
            user_impressions = [news.split("-")[0] for news in impression_entries]

            # Generate rank positions based on CTR
            ranked_positions = rank_submission_format(user_impressions, current_time, past_clicked_articles, past_impressed_articles)

            # Extract clicked and all impression articles from current row
            past_clicked_articles = [news.split("-")[0] for news in impression_entries if news.split("-")[1] == '1']
            past_impressed_articles = (user_impressions, current_time)

            # Write prediction result to file in required format
            f.write(f"{impression_id} {json.dumps(ranked_positions)}\n")

    print(f"✅ Prediction file '{output_file}' successfully created.")


# Run the CTR-based prediction file generation
generate_prediction_file(behaviors_val, output_file="prediction_val_copy.txt")


✅ Prediction file 'prediction_val_copy.txt' successfully created.


# Generate a truth file for evaluation

In [26]:
# Generate ground truth file for validation set
def generate_truth_file(validation_impressions, output_file="truth.txt"):
    """
    Generates a truth.txt file with ground truth click labels.
    """
    with open(output_file, "w") as f:
        for impression_id, news_list in validation_impressions.items():
            labels = [int(news.split("-")[1]) for news in news_list]  # Click labels
            f.write(f"{impression_id} {json.dumps(labels)}\n")  # Format output

    print(f"✅ Truth file '{output_file}' successfully created.")

generate_truth_file(behaviors_val.set_index('ImpressionId')['Impressions'].apply(lambda x: x.split()), output_file="truth_val.txt")

✅ Truth file 'truth_val.txt' successfully created.
